# Sheet 02
This is the implementation of Sheet 02 of Awais Ahmed and Dominic Nieder.

In [ ]:
import numpy as np

* creating "Particle" class, containing all the information of a particle position, velocity and interaction potential (all the object of class "Particle" interact by a Lennard-Jones Potential)
* The "Particle" class contains all particle positions and velocities

In [ ]:
class Particle:
    def __init__(
            self,
            m:float|int,
            x:np.ndarray[float|int],
            y:np.ndarray[float|int],
            vx:np.ndarray[float|int],
            vy:np.ndarray[float|int]
            ):
        self.m:np.ndarray[float|int]=m
        self.x:np.ndarray[float|int]=x
        self.y:np.ndarray[float|int]=y
        self.vx:np.ndarray[float|int]=vx
        self.vy:np.ndarray[float|int]=vy
        
    def __repr__(self):
         return(str("This is a particle at %0.2f, %0.2f with v=%0.2f,%0.2f interacts by Lennard Jones potential" % (self.x,self.y,self.vx,self.vy)) )

### Lennard-Jones potential & force
def lJ_potentil(r12:float|int) -> float:
    c_12 = 9.847044*10**(-6)
    c_6 = 6.2647225*10**(-3)
    return c_12/r12**12-c_6/r12**6

def lj_force(r12:float|int) -> float:
    c_12 = 9.847044*10**(-6)
    c_6 = 6.2647225*10**(-3)
    return (12*c_12/r12**13-6*c_12/r12**7)

* the following functions allow for an appropriat nearest neighbour search

In [ ]:
### calculating shift to S->S' of which p1'=(box[0]/2, box[1]/2)
def system_shift(
        x:float,
        y:float,
        box:tuple[float|int]
        ) -> tuple[float]:
    (dx,dy) = (x-box[0]/2 , y-box[1]/2)
    return(dx,dy)

### vec(p2->p1)
def rel_koordiantes(
        x1:float,
        y1:float,
        x2:float,
        y2:float,
        box:tuple[float|int]
        ) -> tuple[float|int]:  
    dx, dy = system_shift(x1,y1,box)  # S->S': S' is a box with p1' at its center
    return(x1 +dx - (x2-dx)%box[0] , y1 +dy - (y2-dy)%box[1])


The follow up code can be used to select data to analyse

In [ ]:
### calculating distributions
### |v| distribution
def collect_vabs(vx:list[list[float|int]],vy:list[list[float|int]],interval:tuple[int]) -> list[float]:
    v_abs=np.zeros(len(vx)+(interval[1]-interval[0]))
    for i in range(len(vx)):
        for j in range(interval[0],interval[1]):
            v_abs[i+j-interval[0]]=(np.sqrt(vx[i][j]**2+vy[i][j]**2))
    return v_abs

### velocity x or y distribution
def collect_v_1D(vx:list[float|int],interval:tuple[int]) -> list[float]:
    l=np.zeros(len(vx)+(interval[1]-interval[0]))
    print(len(l))
    for i in range(len(vx)):
        for j in range(interval[0],interval[1]):
            l[i+j-interval[0]]=(vx[i][j])
    return l


In [ ]:
def integrate(
        x:float,
        y:float,
        vx:float,
        vy:float,
        ax:float,
        ay:float,
        dt:float|int,
        box:tuple[float|int]
        ) -> tuple[float]:
    dx = vx*dt+ax*dt*dt/2
    dy = vy*dt+ay*dt*dt/2
    return (x+dx)%box[0] , (y+dy)%box[1]


def integrate(v,a1,a2,dt):
    dv = 1/2 (a1+a2) *dt
    return v+dv


def iterate(
        dt:float|int,
        n:int,
        particles:list[Particle],
        box:tuple[float|int],
        data:list
        ):
    for i in range(n):
        ### collision
        for j in range(i+1,n):  # iteration over all others
            d_vec:tuple[float|int] = rel_koordiantes(particles[i],particles[j], box)  # rel. distance vector j -> i
            d_abs = d_vec[0]**2+d_vec[1]**2  # distance squared
            #### collision condition -> kinematics
            if np.sqrt(d_abs) <= particles[i].r+particles[j].r and d_abs != 0:
                d_unit:tuple[float|int] = d_vec/np.sqrt(d_abs)  # collision axis
                dvix , dviy= -((particles[i].vx-particles[j].vx)*d_unit[0]+ (particles[i].vy-particles[j].vy)*d_unit[1]) *d_unit
                particles[i].vx , particles[i].vy = (particles[i].vx+dvix) , ( particles[i].vy+dviy)
                particles[j].vx , particles[j].vy = (particles[j].vx-dvix) , (particles[j].vy-dviy)
        integrate()

    return data    




In [ ]:
def simulate(
        particles:list[Particle], 
        n:int, 
        T:int, 
        dt:float|int, 
        box:tuple[float|int],
        data:list
        ):
    position=np.zeros()
    for t in range(T):
        iterate(dt,n,particles,box,position, velocity)
    return data

In [ ]:
x_data=np.zeros((n,T))
y_data=np.zeros((n,T))
vx_data=np.zeros((n,T))
vy_data=np.zeros((n,T))
